In [1]:
#Imports
import cv2
import mxnet as mx
import pandas as pd
import numpy as np
import time
import re
import os
from geopy.distance import geodesic
from sklearn.preprocessing import normalize
from tqdm import tqdm
import torch
from datetime import datetime
import sys
import warnings
warnings.filterwarnings("ignore")
import glob
sys.path.append("./FaceDetector/")
from FaceDetector.detect_functions import inference,preprocess

remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456
Model Loaded


In [2]:
#Embedding Model building
def get_model(ctx, image_size, model_str, layer):
    prefix = "./FaceRecognition/model-r100-ii/model"
    epoch = 0
    print('loading',prefix, epoch)
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    all_layers = sym.get_internals()
    sym = all_layers[layer+'_output']
    model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)
    model.bind(data_shapes=[('data', (1, 3, image_size[0], image_size[1]))])
    model.set_params(arg_params, aux_params)
    return model

In [3]:
#Configs
sim_thresh=0.30
det_thresh=0.85
km_thresh=5 #search cameras 5KM radius
#Initiliaze
df_cameras=pd.read_csv("../CameraFeed/camera.csv")
x=torch.rand((800,800,3))
inference(x)
embedding_model = get_model(mx.gpu(), (112, 112), None, 'fc1')

loading ./FaceRecognition/model-r100-ii/model 0


In [4]:
#Find if person is found using detections
def findmatch(personE,dets,imgO):
    imgc=imgO.copy()
    found=False
    for b in dets:
        if b[4] < det_thresh:
            continue
        r1=max(0,int(b[1]))
        r2=min(imgO.shape[0],int(b[3]))
        c1=max(0,int(b[0]))
        c2=min(imgO.shape[1],int(b[2]))
        cropimg=imgO[r1:r2,c1:c2]
        if cropimg.shape[0]==0 or cropimg.shape[1]==0:
            continue

        #Recogniton
        #Align it
        align_image=preprocess(imgO,b[0:4],b[5:].reshape(5,2),image_size='112,112')
        person_detectedE=get_feature(align_image)
        sim=abs(np.dot(personE,person_detectedE))
        if sim>sim_thresh:
            found=True
            cv2.rectangle(imgc, (b[0], b[1]), (b[2], b[3]), (255, 0, 0), 2)
    if found:
        return imgc
            
#todatetime
def str2datetime(s):
    c=np.array(re.split('[- : _]',s)).astype('int')
    c=datetime(c[2],c[1],c[0],c[3],c[4])
    return c

#Get the camera feed on specific interval
def getcameraI(cameraF,time_period):
    lower=str2datetime(time_period[0])
    higher=str2datetime(time_period[1])
    cameraAll=glob.glob(cameraF+'/*')
    cameraSI=[]
    for cpath in cameraAll:
        c=cpath.split("/")[-1].split(".")[0]
        c=str2datetime(c)
        if c>=lower and c<higher:
            cameraSI.append(cpath)
    return cameraSI

#Searches the camera feed on certrain intervals
def searchcameraF(case_id,personE,cameraF,time_period):
    cameraI=getcameraI(cameraF,time_period)
    
    #For each video clip selected from one camera
    cameraid=str(cameraF.split("/")[-2])
    index=0
    for camera in cameraI:
        cap = cv2.VideoCapture(camera)
        total_frames= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        success, frame = cap.read()
        loop=tqdm(total=total_frames)
        while(success):
            frame=frame[:,:,::-1]
            dets=inference(frame)
            img_output=findmatch(personE,dets,frame.copy())
            if img_output is not None:
                cv2.imwrite("../Case/99/snapshots/{}_{}.jpg".format(cameraid,str(index)),img_output[:,:,::-1])
                
                
                index+=1
            loop.update(1)
            success, frame = cap.read()

In [5]:
#GetEmbedding of the face
def get_feature(img_aligned):
    aligned = np.transpose(img_aligned, (2,0,1))
    aligned=aligned[np.newaxis,...]
    data = mx.nd.array(aligned)
    db = mx.io.DataBatch(data=(data,))
    embedding_model.forward(db, is_train=False)
    embedding = embedding_model.get_outputs()[0].asnumpy()
    embedding = normalize(embedding).flatten()
    return embedding

In [6]:
#Returns cameras which needed to be searched for the person
def sortedCameras(lat,long,time_period):
    d=[]
    for _,row in df_cameras.iterrows():
        dis=geodesic((lat,long),(row['lat'],row['long'])).kilometers
        d.append(dis)
    tempdf=df_cameras.copy()
    tempdf['dist']=d
    tempdf=tempdf.loc[tempdf['dist']<=km_thresh,:]
    tempdf.sort_values(by='dist',ascending=False,inplace=True)
    return tempdf['id'].values

#Get Embedding of person to search
def getpersonE(image_path):
    img=cv2.imread(image_path)[:,:,::-1]
    imgc=img.copy()
    dets=inference(img)
    b=dets[dets[:,4].argmax(),:]
    cropped_align_face=preprocess(imgc,b[0:4],b[5:].reshape(5,2),image_size='112,112')
    em=get_feature(cropped_align_face)
    return em

In [7]:
#Smartly Search for the person in closest cameras
def smartSearch(case_id,lat,long,time_period):
    person='../Case/{}/picture.jpg'.format(str(case_id))
    os.makedirs("../Case/"+str(case_id)+'/'+'snapshots',exist_ok=True)
    camerasId=sortedCameras(lat,long,time_period)
    cameras2search=df_cameras.loc[df_cameras['id'].isin(camerasId),:]
    #Get Embedding of person to search
    personE=getpersonE(person)
    for _,row in cameras2search.iterrows():
        cameraF='../CameraFeed/'+row['city']+'/'+row['location']+'/'+str(row['id'])
        searchcameraF(case_id,personE,cameraF,time_period)

In [8]:
#HIT
smartSearch(99,33.53,73.16,['20-11-2020_19:10','21-11-2020_18:50'])

 17%|█▋        | 977/5704 [00:37<02:28, 31.92it/s]


 36%|███▌      | 2029/5704 [01:15<03:32, 17.26it/s]


 52%|█████▏    | 2967/5704 [01:53<01:24, 32.58it/s]


 72%|███████▏  | 4087/5704 [02:30<00:49, 32.51it/s]


 92%|█████████▏| 5271/5704 [03:06<00:12, 33.33it/s]


100%|██████████| 5704/5704 [03:34<00:00, 32.39it/s]